In [2]:
import whisper
import wave
import librosa
import soundfile as sf
import sys
import time
import scipy.io.wavfile
from IPython.display import Audio, display
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from jiwer import wer  # Импортируем библиотеку jiwer

def convert_to_wav_librosa(input_file, output_file):
    """Convert the audio file to WAV format with mono PCM encoding using librosa."""
    try:
        # Загрузка аудиофайла и преобразование
        audio, sr = librosa.load(input_file, sr=16000, mono=True)
        
        # Сохранение аудио в формате WAV с помощью soundfile
        sf.write(output_file, audio, sr, subtype='PCM_16')
    except Exception as e:
        print(f"Error during conversion: {e}")
        sys.exit(1)

def csvToWav(pathToCsv, countsWavs):
    df = pd.read_csv(pathToCsv, index_col=0)
    sample_rate = 16000  # частота дискретизации в Гц
    dirToSave = "/home/user/Melkumyan/outputWav/"
    ## читает список в строку - нужен перевод - eval
    for i in range(countsWavs):
        y = eval(df.sound.values[-i-1])
        y_array = np.array(y)  # преобразование списка в массив NumPy
        y_array = y_array * 32767  # нормализация данных в диапазон [-32768, 32767]
        y_array = y_array.astype(np.int16)  # преобразование в целочисленный тип данных
        scipy.io.wavfile.write(dirToSave+f"output_{i}.wav", sample_rate, y_array)
        if (i % 50 == 0):
            print(i)
        # ys = df.speech.values[-i-1]
# print(countsWavs-1)
# Загрузка модели (можно выбрать разные модели: "tiny", "base", "small", "medium", "large")
model = whisper.load_model("large")
dirToOpen = "/run/user/1000/newuser/denoise/outputWav/"
csvToWav('../DataSet/speech_to_text.csv', 769)
print('Готово')


 12%|████▌                                 | 354M/2.88G [00:50<06:10, 7.33MiB/s]


KeyboardInterrupt: 

In [ ]:
print(1)

In [1]:
import pandas as pd
from datasets import Dataset, load_metric
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Trainer, TrainingArguments
import soundfile as sf

dirToOpen = "/run/user/1000/newuser/denoise/outputWav/"
# Подготовка данных: wav файл и его расшифровка
# Предполагается, что у вас есть список аудиофайлов и их текстовых транскрипций
df = pd.read_csv('../DataSet/speech_to_text.csv', index_col=0)
data = []
for i in range(df.shape[0]):
    data.append({"path": dirToOpen+f"output_{i}.wav", "transcription": df.text.values[-i-1]})


df = pd.DataFrame(data, columns=['path', 'transcription'])

# Преобразование данных в формат, подходящий для Dataset
def map_to_dict(batch):
    return {
        'audio': sf.read(batch['path'])[0],
        'transcription': batch['transcription']
    }

dataset = Dataset.from_pandas(df).map(map_to_dict)

# Загрузка процессора и модели Whisper
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")

# Преобразование данных для модели
def preprocess_function(batch):
    audio = batch['audio']
    transcription = batch['transcription']
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    inputs['labels'] = processor(text=transcription, return_tensors="pt", padding=True)['input_ids']
    return inputs

dataset = dataset.map(preprocess_function, batched=True)

# Настройка аргументов для тренировки
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="no",  # Отключение оценки
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Создание тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Запуск тренировки
trainer.train()

# Сохранение модели
model.save_pretrained('./fine_tuned_whisper')
processor.save_pretrained('./fine_tuned_whisper')
print('Готово')

2024-09-17 10:27:49.127768: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-17 10:27:49.129782: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-17 10:27:49.171052: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 10:27:49.793115: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Map:   0%|          | 0/769 [00:00<?, ? examples/s]

Map:   0%|          | 0/769 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: urpok (urpok-urpok). Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/home/user/pythongpu/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
Some non-default generation parameters are set in the model config. These should go into a G

Готово


In [2]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import soundfile as sf
import torch
import pandas as pd
import jiwer
# Загрузка модели и процессора
processor = WhisperProcessor.from_pretrained("./fine_tuned_whisper")
model = WhisperForConditionalGeneration.from_pretrained("./fine_tuned_whisper")

# Убедитесь, что устройство поддерживает CUDA, если это нужно
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
def transcribe_audio(audio_path):
    # Загрузка аудио
    audio_input, _ = sf.read(audio_path)
    
    # Подготовка входных данных для модели
    inputs = processor(audio_input, sampling_rate=16000, return_tensors="pt").input_features
    inputs = inputs.to(device)

    # Генерация текста
    predicted_ids = model.generate(inputs)
    
    # Декодирование текста
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription
dirToOpen = "/home/user/denoise/outputWav/"
# Подготовка данных: wav файл и его расшифровка
# Предполагается, что у вас есть список аудиофайлов и их текстовых транскрипций
df = pd.read_csv('../DataSet/speech_to_text.csv', index_col=0)
wer_values = []
# Пример использования
for i in range(df.shape[0]):
    audio_file = dirToOpen+f'output_{i}.wav'
    print(f'Запись {i}')
    transcription = transcribe_audio(audio_file)
    print("Транскрипция:", transcription)
    manual_transcription = df.text.values[-i-1]
    print("Ручная транскрипция: ", manual_transcription)
    # Вычисление WER
    wer = jiwer.wer(manual_transcription.lower(), transcription.lower())
    print("WER:", wer)
    wer_values.append(wer)
# Вычисление среднего WER
average_wer = sum(wer_values) / len(wer_values)
print("Среднее WER:", average_wer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Запись 0
Транскрипция:  значит готовый на первый в один центральный 
Ручная транскрипция:  значит готовый на первый в один центральный  
WER: 0.0
Запись 1
Транскрипция: внимание машинист на пути три на пятом километре пекет четыр сработал 
Ручная транскрипция:  внимание машинист на пути три на пятом километре пекет четыр сработал 
WER: 0.0
Запись 2
Транскрипция:  Подходим к депо 
Ручная транскрипция:  подходим к депо 
WER: 0.0
Запись 3
Транскрипция: внимание машинист на пути пять на пятом  
Ручная транскрипция:  внимание машинист на пути пять на пятом  
WER: 0.0
Запись 4
Транскрипция:  он не работает сегодня который сидит на пропадовании  вижу что бы сделать чтоб работать с данными можно было  
Ручная транскрипция:  у него работали все время препадователи  вижу что бы сделать чтоб работать с данными можно было  
WER: 0.5
Запись 5
Транскрипция:  Он шестьдесят девять по швеции правый по двадцать шестой поезд 
Ручная транскрипция:  м сто девять по швеции правый по двадцать шестой поезд 
W

In [3]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

[]


In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.__version__)
print(tf.test.is_built_with_cuda())

2024-09-17 14:52:48.630398: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-17 14:52:48.632516: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-17 14:52:48.673502: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-17 14:52:48.675589: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 14:52:49.440093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Num GPUs Available:  0
2.13.1
True


2024-09-17 14:52:50.539163: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import soundfile as sf
import torch
import pandas as pd
import jiwer
# Загрузка модели и процессора
dir_model = '../анализ эмоциональности/APP_ASR_Emotion_v3/pretrained_models/speech_model/'
processor = WhisperProcessor.from_pretrained(dir_model + "fine_tuned_whisper")
model = WhisperForConditionalGeneration.from_pretrained(dir_model + "fine_tuned_whisper")

# Убедитесь, что устройство поддерживает CUDA, если это нужно
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to('cpu')
def transcribe_audio(audio_path):
    # Загрузка аудио
    audio_input, _ = sf.read(audio_path)
    
    # Подготовка входных данных для модели
    inputs = processor(audio_input, sampling_rate=16000, return_tensors="pt").input_features
    inputs = inputs.to('cpu')

    # Генерация текста
    predicted_ids = model.generate(inputs)
    
    # Декодирование текста
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription



transcribe_audio('/home/boss/Рабочий стол/Данные РЖД/М-Яр/МРС Осадка/04000800_A04.wav')

OSError: Can't load feature extractor for '../анализ эмоциональности/APP_ASR_Emotion_v3/pretrained_models/speech_model/fine_tuned_whisper'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '../анализ эмоциональности/APP_ASR_Emotion_v3/pretrained_models/speech_model/fine_tuned_whisper' is the correct path to a directory containing a preprocessor_config.json file